# Crop the Planet Image

This notebook shows how to crop the planet image.

In [ ]:
import geopandas as gpd
import os
import rasterio
import rasterio.mask
from pathlib import Path
from rasterio.plot import show
from rasterio.crs import CRS
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import numpy as np

# Parameters

Specify *exactly* one. The `site_name` or the `planet_id`. The former is given to the chip by the validation team. Because we are not selecting multiple planet scenes per chip and not selecting planet images that cover multiple chips (they are sufficiently spaced apart), this should be a 1 to 1 mapping.

In [ ]:
PLANET_ID = ''
SITE_NAME = '2_26'

# ^ is exclusive or; makes sure you only specified one
assert((len(PLANET_ID) == 0) ^ (len(SITE_NAME) == 0))

# Environment

In [ ]:
os.environ["AWS_NO_SIGN_REQUEST"] = "YES"

# Get Image Database

In [ ]:
df_images = gpd.read_file('s3://opera-calval-database-dswx/image.geojson')
df_images.dropna(inplace=True)
df_images.head()

In [ ]:
df_site = gpd.read_file('s3://opera-calval-database-dswx/site.geojson')
df_site.dropna(inplace=True)
df_site.head()

In [ ]:
cols_to_merge = [col for col in df_images.columns if col != 'geometry']
df_temp = df_images[cols_to_merge]
df_chips = pd.merge(df_site, df_temp , on='site_name', how='left')
df_chips.head()

In [ ]:
df_chips.buffer(2).plot()

In [ ]:
temp = df_chips[['image_name', 'site_name']]
df_site2image = temp.set_index('site_name')
df_image2site = temp.set_index('image_name')
df_site2image.head()

In [ ]:
if not PLANET_ID:
    values = PLANET_ID = df_site2image.loc[SITE_NAME].tolist()
    PLANET_ID = values[0]
    print(f'There was {len(values)} planet images for this chip')
else:
    values = df_image2site.loc[PLANET_ID].tolist()
    SITE_NAME = values[0]
    print(f'There were {len(values)} chips for this planet_image')

(SITE_NAME, PLANET_ID)

In [ ]:
df_chips.to_file('chips.geojson', driver='GeoJSON')

# Crop

In [ ]:
data_dir = Path(f'data/{PLANET_ID}/')
data_dir.mkdir(exist_ok=True, parents=True)

cropped_dir = Path(f'planet_images_cropped/{PLANET_ID}/')
cropped_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
n = len(PLANET_ID)
planet_images = list(data_dir.glob('*.tif'))
planet_image_path = list(filter(lambda x: x.name[:n] == PLANET_ID, planet_images))[0]
planet_image_path

In [ ]:
with rasterio.open(planet_image_path) as ds:
    planet_crs = ds.crs
    planet_profile = ds.profile
    
planet_profile

In [ ]:
df_chip = df_chips[df_chips.image_name == PLANET_ID]

# 500 meter buffer
df_chip_utm = df_chip.to_crs(planet_crs).buffer(500, join_style=2)
df_chip_utm.plot()

In [ ]:
with rasterio.open(planet_image_path) as src:
    out_image, out_transform = rasterio.mask.mask(src, df_chip_utm.geometry, crop=True)
    out_meta = src.meta

out_meta.update({"driver": "GTiff",
         "height": out_image.shape[1],
         "width": out_image.shape[2],
         "transform": out_transform})

with rasterio.open(cropped_dir / f'cropped_{PLANET_ID}.tif', "w", **out_meta) as dest:
    dest.write(out_image)

Copy original data (if you want), say in the event you are reviewing an old extent.

In [ ]:
if True:
    shutil.copy(str(planet_image_path),
                str(cropped_dir / f'orig_{PLANET_ID}.tif'))

Make new raster for hand-editing

In [ ]:
if True:
    hand_edited = np.zeros(out_image[0,...].shape, dtype='uint8')
    p_he = out_meta.copy()
    p_he['count'] = 1
    p_he['dtype'] = 'uint8'
    p_he['nodata'] = 255
    with rasterio.open(cropped_dir / f'hand_edited_{PLANET_ID}.tif', "w", **p_he) as dest:
        dest.write(hand_edited, 1)

# Check

In [ ]:
fig, ax = plt.subplots()
show(out_image[0,...], transform=out_transform, ax=ax)
df_chip_utm.boundary.plot(ax=ax)